In [1]:
#https://github.com/facebookresearch/poincare-embeddings/tree/main/hype/manifolds
#for poincare embeddings
#maybe look at https://github.com/drewwilimitis/hyperbolic-learning
#maybe: https://lars76.github.io/2020/07/24/implementing-poincare-embedding.html
#maybe: https://pypi.org/project/hyperlib/
#maybe this is better: https://github.com/leymir/hyperbolic-image-embeddings/blob/master/hyptorch/nn.py
#more  about python classes: https://jovian.ai/aakashns/03-logistic-regression#C50
#https://www.w3schools.com/python/python_classes.asp

#CnnEmbedding.ipynb
#My CNN (embedded in hyp.space) for attributes + Knn
#it works but...-----anyway: still not know if ot is good, because I define args.dim and other........??????????
#train data is too big to compute here
#https://github.com/leymir/hyperbolic-image-embeddings/blob/master/examples/mnist.py

from __future__ import print_function

import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import NN as hypnn

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets, neighbors, metrics 
from sklearn.model_selection import cross_val_score

In [2]:
#define a CNN
class Net(nn.Module):
    def __init__(self, args):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 600)  #here were self.fc2 = nn.Linear(500, args.dim)
        self.tp = hypnn.ToPoincare(c=-1, train_x=600, train_c=600, ball_dim=600)
        #up were self.tp = hypnn.ToPoincare(c=args.c, train_x=args.train_x, train_c=args.train_c, ball_dim=args.dim)
        self.mlr = hypnn.HyperbolicMLR(ball_dim=600, n_classes=10, c=-1)
        # Define proportion or neurons to dropout
        self.dropout = nn.Dropout(0.5)         # Apply dropout

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)        # Apply dropout
        x = self.fc2(x)
        x = self.tp(x)
        return x #F.log_softmax(self.mlr(x, c=self.tp.c), dim=-1)   #maybe only return x, then X=np.array(x)??

In [3]:
#LOADING DATASET---mnist dataset


train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

In [4]:
#MAKING DATASET ITERABLE---.....

batch_size = 100
"""
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)
"""
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [5]:
print(len(train_loader))
print(len(test_loader))
print(train_dataset[0])

600
100
(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [7]:
modelH = Net(10)
modelH.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=600, bias=True)
  (tp): ToPoincare(
    c=Parameter containing:
    tensor([-1.], requires_grad=True), train_x=600
  )
  (mlr): HyperbolicMLR(Poincare ball dim=600, n_classes=10, c=-1)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [8]:
#extract the attributes----------------
# Select the desired layer---in this ex. this is fc2
layer = model._modules.get('dropout')
outputs = []

#copy features from penultimate layer (model, input, output)
def copy_features(m, i, o):
    o = o[:, :].detach().numpy().tolist()
    outputs.append(o)
    #o = o[:, :, 0, 0].detach().numpy().tolist() #use this, get error in _ = model(X): IndexError: too many indices for tensor of dimension 2

In [9]:
#Define and attach hook to the penulimate layer
_ = layer.register_forward_hook(copy_features)

In [11]:
# Generate image's features for all images in dloader and saves them in the list outputs
for XX, yy in test_loader:
    _ = model(XX)
print(len(outputs))

100


In [12]:
# flatten list of features to remove batches!!!!!!!!!!!!!!----in numpy array....
list_features = [item for sublist in outputs for item in sublist]
X = np.array(list_features)
print(X.shape)
print(X[0])

(10000, 500)
[0.08901027 0.         0.03132187 0.         0.04969964 0.0692743
 0.         0.08693587 0.         0.         0.14620221 0.
 0.08679606 0.         0.         0.00780601 0.         0.
 0.0797523  0.         0.         0.05608506 0.         0.
 0.         0.0541016  0.         0.         0.         0.
 0.         0.04857452 0.01219959 0.         0.0092326  0.
 0.         0.07752223 0.         0.06060091 0.         0.00043647
 0.         0.         0.         0.01401155 0.         0.00624087
 0.02161368 0.         0.04952208 0.0087075  0.         0.0666145
 0.07864208 0.         0.         0.04007846 0.         0.05858412
 0.12840928 0.         0.0621311  0.         0.04460188 0.
 0.08023936 0.         0.         0.04654349 0.098539   0.
 0.04274815 0.         0.03998552 0.         0.         0.04019527
 0.         0.01041107 0.         0.         0.06077857 0.0866178
 0.00647829 0.11181688 0.         0.03800711 0.         0.
 0.03978224 0.08097462 0.07297568 0.         0.06

In [13]:
train_df = pd.read_csv('mnist_test.csv')
X = np.array(list_features)
y = np.array(train_df.iloc[:, 1])   #all rows & only first column

In [14]:
def poincare_distance(row1, row2):
    #print('len(row1) = ',len(row1))
    #print('len(row1) = ',len(row1))
    distance = 0.0
    d = 0.0
    norm1 = 0.0
    norm2 = 0.0
    
    d = (row1 - row2)*(row1 - row2)
    d = d.sum()
    
    norm1 = row1 * row1
    norm1 = norm1.sum()
    
    norm2 = row2 * row2
    norm2 = norm2.sum()
    
    m = 1 + 2*d/((1-norm1)*(1-norm2))
    distance = np.log(m + np.sqrt(m*m - 1))
    
    return(distance)

In [ ]:
k_scoresP = []
for k in range(1,5):
    knn = neighbors.KNeighborsClassifier(n_neighbors=k, leaf_size=3, metric='pyfunc', metric_params={"func":poincare_distance}, n_jobs=3)	#to add new metric: metric='pyfunc'
    #for testing
    #knn = neighbors.KNeighborsClassifier(n_neighbors=5, algorithm='brute', metric='euclidean')    #algorithm='ball_tree' - for a custom metric
    #fit the model
    knn.fit(X,y)
    #cross validation-------------------------------after knn
    scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
    k_scoresP.append(scores.mean())
print(k_scoresP)

/home/alenkat/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in log
/home/alenkat/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in log


In [ ]:
print("Maximum accuracy:",max(k_scoresEM),"at k =",k_scoresEM.index(max(k_scoresEM))+1)